In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import ubiquant
from distutils.dir_util import copy_tree

copy_tree("../input/nbeats-keras/nbeats_keras", "./nbeats_keras") # copy into nbeats working directory
from nbeats_keras.model import NBeatsNet as NBeatsKeras

### Data prep

In [ ]:
df = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet').astype('float16') # convert to float16 to prevent out of memory
f_col = df.drop(['row_id','time_id','investment_id','target'],axis=1).columns
df.head()

In [ ]:
f_col

In [ ]:
df_y = pd.DataFrame(df['target'])
df_x = df[f_col]
del df #free up memory

In [ ]:
x, y = df_x.values, df_y.values
num_samples, time_steps, input_dim, output_dim = len(x), len(x[0]), 1, 1
print(num_samples, time_steps)

### NBeats Model

In [ ]:
backend = NBeatsKeras(
            backcast_length=time_steps, forecast_length=output_dim,
            stack_types=(NBeatsKeras.GENERIC_BLOCK, NBeatsKeras.GENERIC_BLOCK),
            nb_blocks_per_stack=4, thetas_dim=(4, 4), share_weights_in_stack=True,
            hidden_layer_units=512
        )

backend.compile(loss='mae', optimizer='adam')

c = num_samples // 10 # 10% for validation
x_train, y_train, x_test, y_test = x[c:], y[c:], x[:c], y[:c]

In [ ]:
backend.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=1, batch_size=1024)

### Submission

In [ ]:
env = ubiquant.make_env()   
iter_test = env.iter_test()    
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df[f_col]
    sample_prediction_df['target'] = np.squeeze(backend.predict(test_df))
    env.predict(sample_prediction_df)